In [ ]:
# Import required libraries
import PIL
import json
!pip install -q streamlit
import streamlit as st
import requests
import pandas as pd
%pip install ultralytics
from ultralytics import YOLO
from collections import defaultdict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 6.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/NutritionExperts/config.json') as file:
    config = json.load(file)

In [ ]:
config_yolo = config['yolo']
config_cn = config['calorieninjas']

In [ ]:
MODEL_PATH = config_yolo['model_path']
CN_API_URL = config_cn['api_url']
CN_API_KEY = config_cn['api_key']

In [ ]:
# Setting page layout
st.set_page_config(
    page_title="Meal Analysis",  # Setting page title
    page_icon='📊'
)

In [ ]:
# Creating main page heading
st.title("Meal Analysis")

In [ ]:
# Adding file uploader to sidebar for selecting images
source_img = st.file_uploader(
    "Upload meal image...", type=("jpg", "jpeg", "png", 'bmp', 'webp'))

In [ ]:
# Creating two columns on the main page
col1, col2 = st.columns(2)

In [ ]:
# Adding image to the first column if image is uploaded
with col1:
    if source_img:
        # Opening the uploaded image
        uploaded_image = PIL.Image.open(source_img)
        # Adding the uploaded image to the page with a caption
        st.image(source_img,
                 caption="Uploaded Image",
                 use_column_width=True
                 )
try:
    model = YOLO(MODEL_PATH)
except Exception as ex:
    st.error(
        f"Unable to load model. Check the specified path: {MODEL_PATH}")
    st.error(ex)
if st.button('Detect Food Items'):
    res = model.predict(uploaded_image)
    boxes = res[0].boxes
    res_plotted = res[0].plot()[:, :, ::-1]
    with col2:
        st.image(res_plotted,
                 caption='Detected Image',
                 use_column_width=True
                 )

    # analyze nutritional content
    names = model.names
    resDict = defaultdict(int)

    for r in res:
        for c in r.boxes.cls:
            resDict[names[int(c)]] += 1

    resDict = dict(resDict)
    foodList = sorted(list(resDict.keys())) # sort to prevent 'rice milk' bug

    # Delete ambigious other foods class from foodList
    if 'Other foods' in foodList:
        foodList.remove('Other foods')

    # st.subheader('Detected Foods: ' + str(foodList))

    # Create query
    query = ''

    for food in foodList:
        query += (food + ', ')

    # Get calorie ninja nutritional info
    response = requests.get(CN_API_URL + query, headers={'X-Api-Key': CN_API_KEY})
    if response.status_code == requests.codes.ok:
        nutritionDict = json.loads(response.text)
    else:
        print("Error:", response.status_code, response.text)

    # display nutritional information as a pandas dataframe table
    nutrition_df = pd.DataFrame.from_records(list(nutritionDict.values())[0])

    with st.expander("Nutrition Facts"):
        st.table(nutrition_df)